In [3]:
import osmnx
import pandas as pd

This notebook takes the network, aggregates the link flows for the operators and outputs a flow network in GeoPackage format that can be visualized using QGIS.

In [ ]:
if "snakemake" in locals():
    network_path = snakemake.input[0]
    flow_paths = snakemake.input[1:]
    output_path = snakemake.output[0]
else:
    network_path = "../../results/network/network.graphml"
    flow_paths = ["../../results/operators/synthetic1/routes.parquet"]
    output_path = "../../results/aggregation/flows.gpkg"

In [8]:
# Load graph
osmnx.io._convert_bool_string = lambda x: bool(x) if not x is None else False
graph = osmnx.load_graphml(network_path)

In [ ]:
# Convert network
df_network = osmnx.graph_to_gdfs(graph, nodes = False, edges = True).reset_index()[["u", "v", "geometry"]].rename(columns = {
    "u": "origin", "v": "destination"
})

In [ ]:
# Load flows
df_flow = pd.concat([
    pd.read_parquet(path) for path in flow_paths
]).groupby(["origin", "destination"])["flow"].sum().reset_index()

In [32]:
# Merge information
df_flow = pd.merge(df_network, df_flow, on = ["origin", "destination"], how = "left")
df_flow["flow"] = df_flow["flow"].fillna(0.0)

In [ ]:
# Output
df_flow[["flow", "geometry"]].to_crs("EPSG:4326").to_file(output_path)